# Emotion-Aware Customer Feedback Analysis with Explainable AI

## Project Objective

Build an end-to-end NLP system that analyzes customer feedback to detect fine-grained emotions, uncover recurring complaint themes, and provide explainable insights through a deployable Streamlit web app, containerized using Docker for reproducibility.

The system combines Transformer-based modeling (DistilRoBERTa), explainable AI (SHAP/LIME), and unsupervised clustering (UMAP + HDBSCAN) — integrated inside a VS Code-friendly development environment with PyTorch optimized for macOS.

## Real-World Scenario

Companies receive thousands of text reviews or feedback entries daily. This project automates emotion detection and insight extraction to help organizations:

- Identify dissatisfaction or delight trends early
- Discover emerging product or service themes
- Explain why customers feel a certain way using interpretable AI

## Technology Stack

| Layer | Tools & Frameworks |
| --- | --- |
| Language / IDE | Python 3.10+, VS Code (dev container ready) |
| Environment | Conda / venv inside Docker container |
| Core Libraries | PyTorch (MPS backend), Transformers, scikit-learn, pandas, numpy |
| NLP Models | distilroberta-base (for emotion classification), all-MiniLM-L12-v2 (for embeddings) |
| Explainability | SHAP, LIME, Attention Visualization |
| Clustering | Sentence-Transformers, UMAP, HDBSCAN |
| Visualization & UI | Streamlit, Plotly |
| Containerization | Docker + Dockerfile + .dockerignore |
| Experiment Tracking (optional) | TensorBoard / Weights & Biases |